In [ ]:
# add h4d proj root to sys.path so we can import stuff that's in h4d_main/h4d:
# Example: suppose you want to import something from <h4d_main>/h4d/h4d/submodule/:
# "import h4d.submodule" or "from h4d.submodule import foo"
import sys, subprocess
sys.path.append(subprocess.check_output(['git','rev-parse','--show-toplevel']).strip().decode("utf-8") )


In [ ]:
from typing import Dict, List, Tuple

## Calculate Num default boxes at each detection layer:

In [ ]:
# Feature maps and layers for SSD300:
feature_maps = [38, 19, 10, 5, 3, 1]
source_layers = {
    "conv4_3": 38
    , "conv7": 19
    , "conv8_2": 10
    , "conv9_2": 5
    , "conv10_2": 3
    , "conv11_2": 1
}

In [ ]:
def get_sizes_sq(layer_names):
    return [source_layers[k]**2 for k in layer_names]

def counts_dp(c: List[int], t: int, feature_sizes: List[int], memo: Dict[Tuple, bool], results, max_count):
    """Dynamic Programming solution to find all box counts that sum to total (t)."""
    # base cases:
    if t==0:
        results.append(c)
    if t<0:
        return
    for idx, f in enumerate(feature_sizes):
        if c[idx] <= max_count:
            c_new = c.copy()
            c_new[idx] += 1
            c_new_tuple = tuple(c_new)
            if c_new_tuple not in memo:
                memo[c_new_tuple] = True
                counts_dp(c_new, t-f, feature_sizes, memo, results, max_count)
                
def get_box_counts(T: int, layers: List, min_count = 3, max_count=16) -> List[int]:
    """
    Finds box counts for the give feature map sizes, such that box counts sum to t.
    Created this to try to reproduce the box counts used to generate figures in 
    page 10 of SSD paper.
    """
    feature_sizes = get_sizes_sq(layers)
    c = [min_count] * len(feature_sizes)
    t = T - (sum(min_count*feature_sizes))
    memo = {}
    results = []      
    counts_dp(c, t, feature_sizes, memo, results, max_count)
    check_results(results, feature_sizes, T)
    return results

def check_results(results, feature_sizes, expected_box_count):
    for counts in results:
        total = box_count(feature_sizes, counts)
        if expected_box_count != total:
            print("total boxes: ", total)

def box_count(feature_sizes, counts):
    total = 0
    for i,c in enumerate(counts):
        total += (c*feature_sizes[i])
    return total

In [ ]:
get_box_counts(8664, ["conv7"], min_count=1, max_count=25)

In [ ]:
get_box_counts(9025, ["conv4_3", "conv7"])

In [ ]:
get_box_counts(9864, ["conv4_3", "conv7", "conv8_2"], min_count=1, max_count=26)

In [ ]:
get_box_counts(8942, ["conv4_3", "conv7", "conv8_2", "conv9_2"])

In [ ]:
get_box_counts(8764, ["conv4_3", "conv7", "conv8_2", "conv9_2", "conv10_2"], min_count=2)

In [ ]:
get_box_counts(8732, ["conv4_3", "conv7", "conv8_2", "conv9_2", "conv10_2", "conv11_2"], min_count=4, max_count=6)

In [ ]:
box_count([38**2, 19**2], [6,6])

## Aspect Ratio Notes:

asdf